In [25]:
% matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
#from sklearn.datasets import load_digits
from sklearn.linear_model import LogisticRegression
from sklearn import svm


import matplotlib.image as mpimg
from skimage.io import imread, imshow



## Opdracht 1 - wine classification en cancer detection via SVM

Doe de wine classification en de cancer detection uit vorige opdracht over met support vector machines. Gebruik hiervoor cross-validation pipeline om de beste parameters te zoeken: lineaire kernel vs polynomial of RBF kernel, beste waarde voor C en Gamma. Vergelijk de resultaten met deze die bekomen werden via logistic regression. Welke presteert het best in termen van accuracy, recall, precision en f1 score? Zoek een verklaring.

In [3]:
#Wine classification

In [6]:
dataframe = pd.read_csv('wine_data.csv')
dataframe.head()

,inputs Alcohol,MalicAcid,Ash,AlcalinityOfAsh,Magnesium,TotalPhenols,flavanoids,NonflavanoidsPhenols,Proanthocyanins,ColorIntensity,Hue,OD280/OD315,Proline,Cultivar
0,12.08,1.33,2.30,23.6,70,2.20,1.59,0.42,1.38,1.74,1.07,3.21,625,1
1,12.08,1.13,2.51,24.0,78,2.00,1.58,0.40,1.40,2.20,1.31,2.72,630,1
2,12.37,1.17,1.92,19.6,78,2.11,2.00,0.27,1.04,4.68,1.12,3.48,510,1
3,13.11,1.01,1.70,15.0,78,2.98,3.18,0.26,2.28,5.30,1.12,3.18,502,1
4,12.04,4.30,2.38,22.0,80,2.10,1.75,0.42,1.35,2.60,0.79,2.57,580,1


In [16]:
#aantal klasses: 
aantal = dataframe.Cultivar.unique()
print(aantal)

#gebalanceerd? 

g = dataframe.groupby('Cultivar')
g.Cultivar.count()



[1 2 0]


Cultivar
0    59
1    71
2    48
Name: Cultivar, dtype: int64

In [17]:
#opsplitsen in feautres & targets 
features = list(dataframe.columns[0:dataframe.columns.size-1])
X = dataframe[features].values
y = dataframe.Cultivar.values

In [20]:
#normaliseren
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

In [21]:
#opsplitsen in train & test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

In [26]:
#aanmaken van het model + gridsearch op toepassen (croos validation!)
from sklearn.model_selection import GridSearchCV


model = svm.SVC()
paramaters = [ 
        {'kernel': ['linear'], 'C': np.linspace(1,20,100)},
        {'kernel': ['rbf'], 'C': [1, 10], 'gamma': [0.0001, 0.001, 0.01, 0.1, 0.2]},
        {'kernel': ['poly'], 'C':[1, 10]} ]
grid_search = GridSearchCV(estimator = model, 
                           param_grid = paramaters,
                           scoring = 'accuracy',
                           cv = 4,
                           n_jobs = -1)
grid_search = grid_search.fit(X_train, y_train)

best_accuracy = grid_search.best_score_ 
best_parameters = grid_search.best_params_  

print('Best accuracy : ', grid_search.best_score_)
print('Best parameters :', grid_search.best_params_  )

y_pred = grid_search.predict(X_test)
print(classification_report(y_test, y_pred))

cf = confusion_matrix(y_test, y_pred)
print(cf)
print(accuracy_score(y_test, y_pred) * 100) 

Best accuracy :  0.9788732394366197
Best parameters : {'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}
             precision    recall  f1-score   support

          0       1.00      0.92      0.96        13
          1       0.88      1.00      0.93         7
          2       1.00      1.00      1.00        16

avg / total       0.98      0.97      0.97        36

[[12  1  0]
 [ 0  7  0]
 [ 0  0 16]]
97.22222222222221


In [ ]:
#svm- cancer detection

In [47]:
#inlezen
df = pd.read_csv('cancer.csv')
df.head()
# df.shape

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
1,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
2,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
3,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN
4,843786,M,12.45,15.70,82.57,477.1,0.12780,0.17000,0.1578,0.08089,...,23.75,103.40,741.6,0.1791,0.5249,0.5355,0.1741,0.3985,0.12440,NaN


In [35]:
#omzetten van diagnosis naar een numerieke waarde
df.diagnosis = df.diagnosis.replace('M',1)
df.diagnosis = df.diagnosis.replace('B',0)

In [39]:
# aantal klasses + gebalanceerd? 

aantal = df.diagnosis.unique()

g = df.groupby('diagnosis')
g.diagnosis.count()

diagnosis
0    357
1    211
Name: diagnosis, dtype: int64

In [48]:
#opsplitsen in features & targets
features = df.columns[2:df.columns.size-1]
print(features)
X = df[features].values
y = df.diagnosis.values

Index(['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean',
       'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst'],
      dtype='object')


In [49]:
#normaliseren (dit is noodzakelijk bij het gebruiken van een rbf kernel)

scaler = preprocessing.StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

In [50]:
#opsplitsen in train & testdata
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

In [51]:
#aanmaken van het model + gridsearch
model = svm.SVC()
paramaters = [ 
        {'kernel': ['linear'], 'C': np.linspace(1,20,100)},
        {'kernel': ['rbf'], 'C': [1, 10], 'gamma': [0.0001, 0.001, 0.01, 0.1, 0.2]},
        {'kernel': ['poly'], 'C':[1, 10]} ]
grid_search = GridSearchCV(estimator = model, 
                           param_grid = paramaters,
                           scoring = 'accuracy',
                           cv = 4,
                           n_jobs = -1)
grid_search = grid_search.fit(X_train, y_train)

best_accuracy = grid_search.best_score_ 
best_parameters = grid_search.best_params_  

print('Best accuracy : ', grid_search.best_score_)
print('Best parameters :', grid_search.best_params_  )

y_pred = grid_search.predict(X_test)
print(classification_report(y_test, y_pred))

cf = confusion_matrix(y_test, y_pred)
print(cf)
print(accuracy_score(y_test, y_pred) * 100) 

Best accuracy :  0.9823788546255506
Best parameters : {'C': 1.0, 'kernel': 'linear'}
             precision    recall  f1-score   support

          B       0.96      0.99      0.97        71
          M       0.98      0.93      0.95        43

avg / total       0.97      0.96      0.96       114

[[70  1]
 [ 3 40]]
96.49122807017544


## Opdracht 2 - MNIST
De MNIST ("Modified National Institute of Standards and Technology")is een veelgebruikte dataset voor het testen en benchmarken van klassificatie algoritmes. Het bevat tienduizenden afbeeldingen van handgeschreven getallen. 
Meer info over deze dataset is te vinden op: http://yann.lecun.com/exdb/mnist/


Bij deze opdracht worden 3 databestanden meegeleverd:
- 'train.csv' bevat gelabelde data voor **trainen** van  de classifier.
- 'test.csv' bevat gelabelde data voor het **testen** van de getrainde classifier
- 'test_Kaggle.csv' bevat niet-gelabelde data voor het evalueren van de classifier via de competitie te vinden op https://www.kaggle.com/c/digit-recognizer. Deze data is dus enkel bruikbaar bij deelname aan de Kaggle competitie. 


1) Train zowel een logistic regression classifier als een Support Vector Machine (met of zonder kernel. Ga via tuning van de hyperparameters op zoek naar de meest performante classifier. Doe dit door de accuracy te maximaliseren of de error rate te minimaliseren op de test.csv dataset. De error rate = 1- accuracy. Het trainen van Support Vector Machines (zeker deze met kernel) vragen enorm veel rekenkracht. Het is daarom verstandig om in eerste instantie te trainen op een klein deel van de training set. Het trainen via logistic regression is minder belastend voor de CPU, desalniettemin wordt aangeraden om de lbfgs solver te gebruiken. *(LogisticRegression(multi_class='multinomial', solver='lbfgs'))*

2) Verzorg de code telkens van commentaar en schrijf jouw conclusies en besluiten neer.

3) Test jouw uiteindelijke classifier met een aantal zelf geschreven getallen. Wat zijn de bevindingen? Waarvan hangt classificatienauwkeurigheid af?

4) Optioneel: test op de 'test_Kaggle' dataset en laad de resultaten in het juiste formaat op naar de Kaggle website. Wat is de behaalde score? Vergelijk deze score met de score op http://yann.lecun.com/exdb/mnist/


Het visualiseren van een digit kan met *'plt.imshow(X_train[n].reshape((28, 28)),cmap = 'gray')'*


In [53]:
#inlezen van beide datasets + ze limiteren tot 1K records (anders veeel te veel memory nodig)
df_train = pd.read_csv('train.csv')
df_train = df_train.loc[1:1000]
df_test = pd.read_csv('test.csv')
df_test = df_test.loc[1:1000]

df_train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
#opsplitsen in features & targets
features_train = list(df_train.columns[1:df_train.columns.size])
X_train = df_train[features_train].values 
y_train= df_train['label'].values

features_test = list(df_test.columns[1:df_test.columns.size])
X_test = df_test[features_test].values 
y_test = df_test['label'].values

In [56]:
#normaliseren van de dataset (hebben we nodig bij svm)
scaler = preprocessing.StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)

scaler.fit(X_test)
X_train = scaler.transform(X_train)

C:\Users\vanbe\AppData\Local\Programs\Python\Python36-32\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [57]:
#LOGISTIC REGRESSION
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
paramaters = [
             {'C' : [0.001, 0.01, 0.1, 1, 10, 100, 1000,10000, 100000]}                                       
             ]
grid_search = GridSearchCV(estimator = model, 
                           param_grid = paramaters,
                           scoring = 'accuracy',
                           cv = 4,
                           n_jobs = -1)
grid_search = grid_search.fit(X_train, y_train)

best_accuracy = grid_search.best_score_ 
best_parameters = grid_search.best_params_  

print('Best accuracy : ', grid_search.best_score_)
print('Best parameters :', grid_search.best_params_  )

y_pred = grid_search.predict(X_test)
print(classification_report(y_test, y_pred))

cf = confusion_matrix(y_test, y_pred)
print(cf)
print(accuracy_score(y_test, y_pred) * 100) 

Best accuracy :  0.7722
Best parameters : {'C': 100000}
             precision    recall  f1-score   support

          0       0.77      0.96      0.86       998
          1       0.54      0.99      0.70      1127
          2       0.93      0.68      0.78       968
          3       0.73      0.79      0.76      1059
          4       0.73      0.80      0.76       914
          5       0.97      0.35      0.52       914
          6       0.90      0.84      0.87       951
          7       0.95      0.78      0.85      1066
          8       0.81      0.52      0.63       995
          9       0.70      0.75      0.72      1008

avg / total       0.80      0.75      0.75     10000

[[ 961    8    0    5    0    0   21    0    2    1]
 [   0 1115    1    2    0    0    0    0    9    0]
 [  45  138  654   40   24    0   21    9   35    2]
 [  21   94   37  835    3    2    7    9   25   26]
 [   5   51    2    0  734    0    6    0    2  114]
 [  69  168    3  180   52  324   24    

In [ ]:

#€€€€€€€€€€€€

In [ ]:
#MET SVM
model = svm.SVC()
paramaters = [ 
        {'kernel': ['linear'], 'C': np.linspace(1,20,100)},
        {'kernel': ['rbf'], 'C': [1, 10], 'gamma': [0.0001, 0.001, 0.01, 0.1, 0.2]},
        {'kernel': ['poly'], 'C':[1, 10]} ]
grid_search = GridSearchCV(estimator = model, 
                           param_grid = paramaters,
                           scoring = 'accuracy',
                           cv = 4,
                           n_jobs = -1)
grid_search = grid_search.fit(X_train, y_train)

best_accuracy = grid_search.best_score_ 
best_parameters = grid_search.best_params_  

print('Best accuracy : ', grid_search.best_score_)
print('Best parameters :', grid_search.best_params_  )

y_pred = grid_search.predict(X_test)
print(classification_report(y_test, y_pred))

cf = confusion_matrix(y_test, y_pred)
print(cf)
print(accuracy_score(y_test, y_pred) * 100)